In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.externals import joblib

In [2]:
#postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_visitas.csv')

In [3]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('Sources/test_final_100k.csv')
visitas = pd.read_csv('Sources/visitas_proc.csv').drop('Unnamed: 0', axis=1)
preds = pd.DataFrame()

In [6]:
postulaciones.drop('Unnamed: 0', axis=1, inplace=True)
postulaciones.head()

,idaviso,idpostulante,sepostulo,visito
0,1112248724,NjlD,1.0,2.168527
1,1112286523,ZaO5,1.0,2.168527
2,1112286523,ZaO5,1.0,2.168527
3,1112272060,ZaO5,1.0,2.168527
4,1112272060,ZaO5,1.0,2.168527


In [7]:
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones = postulaciones.merge(avisos, on='idaviso')
#postulaciones = postulaciones.merge(visitas, on=['idaviso','idpostulante'], how='left')
resultados = postulaciones['sepostulo']
postulaciones.drop(['titulo','descripcion','idaviso','idpostulante','sepostulo'],axis=1, inplace=True)
postulaciones.head()

,visito,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,...,NO_DECLARA,TendenciaSexo,prom_edad,sabados,domingos,ingles,3 anos,2 anos,4 anos,5 anos
0,2.168527,37.0,FEM,12,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,5.0,125.0,...,0.0,FEM,30.251142,0,0,0,0,0,0,0
1,2.168527,27.0,MASC,11,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,5.0,125.0,...,0.0,FEM,30.251142,0,0,0,0,0,0,0
2,2.168527,33.0,FEM,12,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,5.0,125.0,...,0.0,FEM,30.251142,0,0,0,0,0,0,0
3,2.168527,33.0,FEM,12,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,5.0,125.0,...,0.0,FEM,30.251142,0,0,0,0,0,0,0
4,2.168527,26.0,FEM,12,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,5.0,125.0,...,0.0,FEM,30.251142,0,0,0,0,0,0,0


In [8]:
test = test.merge(postulantes, on='idpostulante')
test = test.merge(avisos, on='idaviso')
test = test.merge(visitas, on=['idaviso','idpostulante'], how='left')
test.drop(['titulo','descripcion','idaviso','idpostulante'],axis=1, inplace=True)
test.head()

,id,fechanacimiento,sexo,nivel,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,FEM,...,TendenciaSexo,prom_edad,sabados,domingos,ingles,3 anos,2 anos,4 anos,5 anos,visito
0,0,42.0,FEM,21,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,...,NaN,NaN,0,0,1,0,0,0,0,NaN
1,1,31.0,MASC,9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,...,NaN,NaN,0,0,1,0,0,0,0,NaN
2,2,36.0,FEM,12,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,102.0,NaN,...,NaN,NaN,0,0,1,0,0,0,0,NaN
3,3,69.0,MASC,15,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN
4,4,32.0,FEM,11,Full-time,Otro,Tecnologia / Sistemas,TeleTech,27.0,NaN,...,NaN,NaN,0,0,0,0,0,0,0,NaN


In [9]:
test.drop('id',axis=1,inplace=True)

In [10]:
postulantes = None
avisos = None
visitas = None

In [11]:
test = pd.concat([test,postulaciones])

In [12]:
test['fechanacimiento'].fillna(np.nanmean(test['fechanacimiento']),inplace=True)
test['FEM'].fillna(np.nanmean(test['FEM']),inplace=True)
test['MASC'].fillna(np.nanmean(test['MASC']),inplace=True)
test['NO_DECLARA'].fillna(np.nanmean(test['NO_DECLARA']),inplace=True)
test['prom_edad'].fillna(np.nanmean(test['prom_edad']),inplace=True)
test.loc[:,['sabados','domingos','ingles','3 anos','2 anos','4 anos','5 anos', 'visito']] = test.loc[:,['sabados','domingos','ingles','3 anos','2 anos','4 anos','5 anos', 'visito']].fillna(0)
test.head()

,2 anos,3 anos,4 anos,5 anos,FEM,MASC,NO_DECLARA,TendenciaSexo,cluster,denominacion_empresa,...,fechanacimiento,ingles,nivel,nivel_laboral,nombre_area,prom_edad,sabados,sexo,tipo_de_trabajo,visito
0,0,0,0,0,1286.369363,752.736986,29.780158,NaN,102.0,BUMERAN.COM ARGENTINA,...,42.0,1,21,Jefe / Supervisor / Responsable,Comercial,29.90184,0,FEM,Full-time,0.0
1,0,0,0,0,1286.369363,752.736986,29.780158,NaN,102.0,BUMERAN.COM ARGENTINA,...,31.0,1,9,Jefe / Supervisor / Responsable,Comercial,29.90184,0,MASC,Full-time,0.0
2,0,0,0,0,1286.369363,752.736986,29.780158,NaN,102.0,BUMERAN.COM ARGENTINA,...,36.0,1,12,Jefe / Supervisor / Responsable,Comercial,29.90184,0,FEM,Full-time,0.0
3,0,0,0,0,1286.369363,752.736986,29.780158,NaN,27.0,TeleTech,...,69.0,0,15,Otro,Tecnologia / Sistemas,29.90184,0,MASC,Full-time,0.0
4,0,0,0,0,1286.369363,752.736986,29.780158,NaN,27.0,TeleTech,...,32.0,0,11,Otro,Tecnologia / Sistemas,29.90184,0,FEM,Full-time,0.0


In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not test[col].dtype == 'O': continue
    print(col)
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])

TendenciaSexo
denominacion_empresa
nivel_laboral
nombre_area
sexo
tipo_de_trabajo


In [14]:
test.fillna(-1,inplace=True)
postulaciones = test[100000:]
test = test[:100000]

In [15]:
partitions = []
partitions_res = []
n = 5
index = int(postulaciones.shape[0]/n)
for i in range(n):
    partitions.append(postulaciones[index*i:index*(i+1)])
    partitions_res.append(resultados[index*i:index*(i+1)])


In [16]:
def proba(result):
    prob_1 = []
    for x in result:
        prob_1.append(x[1])
    return prob_1

In [ ]:
print('Arranca XG')

aviso = None
postulantes = None
import xgboost as xgb
dtrain = xgb.DMatrix(data=partitions[0],label=partitions_res[0])
dtest = xgb.DMatrix(data=partitions[4])
param = {'max_depth':15, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
bst = xgb.train(param, dtrain, num_round)
preds['xgb'] = bst.predict(dtest)
print('Termino XG')

In [ ]:
print('Arranca RF')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clf=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
clf.fit(partitions[1],partitions_res[1])
preds['RF'] = proba(clf.predict_proba(partitions[4]))

print('Termino RF')

In [ ]:
print('Arranca NB')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(partitions[2] ,partitions_res[2])
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
preds['NB'] = proba(nb.predict_proba(partitions[4]))

print('Termino NB')

In [ ]:
print('Arranca KNN')

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=25 ,n_jobs=-1, algorithm='auto')
neigh.fit(partitions[3], partitions_res[3])
preds['KNN'] = proba(neigh.predict_proba(partitions[4]))

print('Termino KNN')

In [ ]:
preds.to_csv('Sources/predicciones.csv')

In [17]:
preds = pd.read_csv('Sources/predicciones.csv').drop('Unnamed: 0', axis=1)

In [23]:
#preds.drop('PERCEPTRON',axis=1,inplace=True)
partitions_res[4].shape

(2413411,)

In [ ]:
print('Arranca PERCEPTRON')
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(preds, partitions_res[4])
#from sklearn import svm
#clf = svm.SVC()
#clf.fit(preds, partitions_res[4])

print('PERCEPTRON ya esta entrenado')

In [ ]:
preds.head()
final = pd.DataFrame()
test_final = pd.read_csv('Sources/test_final_100k.csv')['id']

In [ ]:
print('Arranca XGB final')

aviso = None
postulantes = None
import xgboost as xgb
dtrain = xgb.DMatrix(data=postulaciones, label=resultados)
dtest = xgb.DMatrix(data=test)
param = {'max_depth':15, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
bst = xgb.train(param, dtrain, num_round)
final['xgb'] = bst.predict(dtest)
test_final['xgb'] = final['xgb']
print('Termino XGB final')

In [ ]:
print('Arranca RF final')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clfr=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
clfr.fit(postulaciones,resultados)
final['RF'] = proba(clfr.predict_proba(test))
test_final['RF'] = final['RF']
print('Termino RF final')

In [ ]:
print('Arranca NB final')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(postulaciones, resultados)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
final['NB'] = proba(nb.predict_proba(test))
test_final['NB'] = final['NB']
print('Termino NB final')

In [ ]:
print('Arranca KNN final')

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=25 ,n_jobs=-1, algorithm='auto')
neigh.fit(postulaciones,resultados)
final['KNN'] = proba(neigh.predict_proba(test))
test_final['KNN'] = final['KNN']

print('Termino KNN final')

In [ ]:
final.head()

In [ ]:
test_final.to_csv('Sources/TEST_FINAL')

In [ ]:
print('Predigo con Perceptron')

resultado = clf.predict(final)

print('Termino Perceptron')

In [ ]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = resultado
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_pipe_final', index=False)

In [ ]:
from sklearn.externals import joblib
neigh = joblib.load('Sources/model_KNN.pkl') 
nb = joblib.load('Sources/model_NB.pkl')
bst = joblib.dump('Sources/model_XGB.pkl') 
clfr = joblib.dump('Sources/model_RF.pkl') 
final = pd.DataFrame()

In [ ]:
dtest = xgb.DMatrix(data=test)
final['xgb'] = bst.predict(dtest)
final['RF'] = proba(clfr.predict_proba(test))
final['NB'] = proba(nb.predict_proba(test))
final['KNN'] = proba(neigh.predict_proba(test))

In [ ]:
print('Predigo con Perceptron')

resultado = clf.predict_proba(final)

print('Termino Perceptron')